# EVA
## First create a COCO dataformat dataset

In [ ]:
import os
import json
import cv2
import numpy as np
from glob import glob

def create_annotation(points, image_id, category_id, annotation_id):
    segmentation = [p for sublist in points for p in sublist]
    # bounding box calculation
    bbox = [min(points, key=lambda x:x[0])[0], min(points, key=lambda x:x[1])[1], max(points, key=lambda x:x[0])[0]-min(points, key=lambda x:x[0])[0], max(points, key=lambda x:x[1])[1]-min(points, key=lambda x:x[1])[1]]
    area = cv2.contourArea(np.array(points).astype(np.float32))

    annotation = {
        "segmentation": [segmentation],
        "bbox": bbox,
        "area": area,
        "iscrowd": 0,
        "image_id": image_id,
        "category_id": category_id,
        "id": annotation_id
    }
    return annotation

labelme_path = "../strawberry/train/"
annotation_files = glob(os.path.join(labelme_path, "*.json"))

categories_dict = {}
images = []
annotations = []
for i, file in enumerate(annotation_files):
    with open(file) as f:
        data = json.load(f)

    image = {
        "file_name": data['imagePath'],
        "height": data['imageHeight'],
        "width": data['imageWidth'],
        "id": i
    }
    images.append(image)

    for shape in data['shapes']:
        if shape['label'] not in categories_dict:
            categories_dict[shape['label']] = len(categories_dict) + 1
        annotation = create_annotation(shape['points'], i, categories_dict[shape['label']], len(annotations) + 1)
        annotations.append(annotation)

categories = [{"id": value, "name": key} for key, value in categories_dict.items()]

coco_format = {
    "images": images,
    "annotations": annotations,
    "categories": categories
}

with open('../strawberry/output.json', 'w') as f:
    json.dump(coco_format, f)


## Check if torch and cuda version is correct

In [12]:
import torch
print(torch.version.cuda)
print(torch.__version__)

11.1
1.9.0+cu111


In [7]:
import os
os.getcwd()

'/'

In [13]:
!python -m pip install detectron2 -f \
  https://dl.fbaipublicfiles.com/detectron2/wheels/cu111/torch1.9/index.html

Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu111/torch1.9/index.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 40.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.0/131.0 kB 2.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 771.9/771.9 kB 9.8 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: black
    Found existing installation: black 22.3.0
    Uninstalling black-22.3.0:
      Successfully uninstalled black-22.3.0


In [14]:
!python app//EVA/EVA-01/det/tools/lazyconfig_train_net.py \
    --eval-only \
    --config-file app/EVA/EVA-01/det/projects/ViTDet/configs/COCO/cascade_mask_rcnn_vitdet_eva_1536.py \
    "train.init_checkpoint=../models/eva_0356.pth" \
    "model.roi_heads.use_soft_nms=True" \
    'model.roi_heads.method="linear"' \
    "model.roi_heads.iou_threshold=0.6" \
    "model.roi_heads.override_score_thresh=0.0"

Traceback (most recent call last):
  File "app//EVA/EVA-01/det/tools/lazyconfig_train_net.py", line 125, in <module>
    launch(
  File "/usr/local/lib/python3.8/dist-packages/detectron2/engine/launch.py", line 82, in launch
    main_func(*args)
  File "app//EVA/EVA-01/det/tools/lazyconfig_train_net.py", line 109, in main
    cfg = LazyConfig.load(args.config_file)
  File "/usr/local/lib/python3.8/dist-packages/detectron2/config/lazy.py", line 210, in load
    exec(compile(content, filename, "exec"), module_namespace)
  File "app/EVA/EVA-01/det/projects/ViTDet/configs/COCO/cascade_mask_rcnn_vitdet_eva_1536.py", line 4, in <module>
    from .cascade_mask_rcnn_vitdet_b_100ep import (
  File "/usr/local/lib/python3.8/dist-packages/detectron2/config/lazy.py", line 148, in new_import
    exec(compile(content, cur_file, "exec"), module.__dict__)
  File "app/EVA/EVA-01/det/projects/ViTDet/configs/COCO/cascade_mask_rcnn_vitdet_b_100ep.py", line 11, in <module>
    from .mask_rcnn_vitdet_b_100e